# Implementing TF-IDF

Using sklearn, nltk (stopwords)

In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
doc1 = 'they went to the park for bird watching'
doc2 = 'she went to the beach to swim'

We need to convert the text to a vector. First, let's recap the BoW approach:

In [27]:
bow1 = doc1.split(' ')
bow2 = doc2.split(' ')

In [28]:
# use Python's Union function to get unique words:
corpus_unique = set(bow1).union(set(bow2))

Create a dict of words & frequency for each doc in the corpus


In [29]:
num_words_doc1 = dict.fromkeys(corpus_unique, 0)

In [30]:
for word in bow1:
    num_words_doc1[word] += 1

In [31]:
# do the same for `doc2`
num_words_doc2 = dict.fromkeys(corpus_unique, 0)

for word in bow2:
    num_words_doc2[word] += 1

In [32]:
import pandas as pd

In [33]:
corpus_words = {}
for word in corpus_unique:
    corpus_words[word] = [num_words_doc1[word], num_words_doc2[word]]

In [34]:
pd.DataFrame.from_dict(corpus_words)

,the,park,went,bird,watching,she,swim,for,they,beach,to
0,1,1,1,1,1,0,0,1,1,0,1
1,1,0,1,0,0,1,1,0,0,1,2


We need to account for 'stop' word noise

In [35]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

### TF

In [36]:
def compute_term_freq(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [37]:
tf1 = compute_term_freq(num_words_doc1, bow1)
tf2 = compute_term_freq(num_words_doc2, bow2)

### IDF

In [38]:
import math

def compute_IDF(documents):
    N = len(documents)    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict


In [39]:
idfs = compute_IDF([num_words_doc1, num_words_doc2])

In [40]:
idfs

{'the': 0.0,
 'park': 0.6931471805599453,
 'went': 0.0,
 'bird': 0.6931471805599453,
 'watching': 0.6931471805599453,
 'she': 0.6931471805599453,
 'swim': 0.6931471805599453,
 'for': 0.6931471805599453,
 'they': 0.6931471805599453,
 'beach': 0.6931471805599453,
 'to': 0.0}

### TF-IDF

In [41]:
def compute_TFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [42]:
tfidf1 = compute_TFIDF(tf1, idfs)
tfidf2 = compute_TFIDF(tf2, idfs)
df = pd.DataFrame([tfidf1, tfidf2])

In [43]:
df

,the,park,went,bird,watching,she,swim,for,they,beach,to
0,0.0,0.086643,0.0,0.086643,0.086643,0.000000,0.000000,0.086643,0.086643,0.000000,0.0
1,0.0,0.000000,0.0,0.000000,0.000000,0.099021,0.099021,0.000000,0.000000,0.099021,0.0


### Direct implementation with sklearn

In [44]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([doc1, doc2])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [45]:
df

,beach,bird,for,park,she,swim,the,they,to,watching,went
0,0.00000,0.391668,0.391668,0.391668,0.00000,0.00000,0.278675,0.391668,0.278675,0.391668,0.278675
1,0.40698,0.000000,0.000000,0.000000,0.40698,0.40698,0.289569,0.000000,0.579139,0.000000,0.289569


The small difference in values is due to the use of a smoothed version of idf + other optimizations. With a larger doc, the score for `the` would be greatly reduced. [Ref](https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76)


## End to End Example with TF-IDF

In [46]:
# 1. load the dataset as a pandas DataFrame
def load_dataset(filename, text="text", target="target"):
    data = pd.read_csv(filename) # header=None, if needed
    X = data[text]
    y = data[target]
    return X, y

In [47]:
X, y = load_dataset('data/twitter_disaster/train.csv')

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
# 2. test/train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

The next two cells may take about a minute to run

In [50]:
tf_idf_vectorizer = TfidfVectorizer()
X_train_enc = tf_idf_vectorizer.fit_transform(X_train)
X_test_enc = tf_idf_vectorizer.transform(X_test)

## FYI only - not needed for next steps
feature_names = tf_idf_vectorizer.get_feature_names()
dense = X_train_enc.todense()
denselist = dense.tolist()
dense.shape, type(dense)

((5100, 16358), numpy.matrix)

In [61]:
feature_names[:-3]

['00',
 '000',
 '0000',
 '00end',
 '00pm',
 '01',
 '02',
 '0215',
 '02elqlopfk',
 '02pm',
 '03',
 '034',
 '03l7nwqdje',
 '04',
 '05',
 '05th',
 '06',
 '060',
 '061',
 '07',
 '073izwx0lb',
 '08',
 '0840728',
 '087809233445',
 '0880',
 '09',
 '0ajisa5531',
 '0btniwagt1',
 '0bvk5tub4j',
 '0c1y8g7e9p',
 '0cxm5tkz8y',
 '0drqlrsgy5',
 '0dxvz7fdh3',
 '0f8xa4ih1u',
 '0fekgyby5f',
 '0gknpy4lua',
 '0h7oua1pns',
 '0iw6drf5x9',
 '0iyuntxduv',
 '0jfnvaxfph',
 '0jmkdtcymj',
 '0kccg1bt06',
 '0keh2treny',
 '0krw1zyahm',
 '0l',
 '0la1aw9uud',
 '0llwuqn8vg',
 '0lmheaex9k',
 '0lpu0gr2j0',
 '0m1tw3datd',
 '0mcxc68gzd',
 '0npzp',
 '0nr4dpjgyl',
 '0oms8ri3l1',
 '0q040stkcv',
 '0r03c6njli',
 '0rny349unt',
 '0rokdutyun',
 '0rsverlztm',
 '0scnwe8xbv',
 '0szwlwl9qu',
 '0t8vqkeari',
 '0tslqjokvh',
 '0tzsqe8esd',
 '0ubg9wfyge',
 '0uk0h9hozn',
 '0usdi5jcho',
 '0vcsafx9by',
 '0wdazlcm90',
 '0wdseihyqu',
 '0wratka2jl',
 '0x2bieqxpv',
 '0x8jaqtowm',
 '0xa6fcjyec',
 '0xcb7yeqmw',
 '0xouv7dhwz',
 '0xrfvidxa2',
 '0yo2l3

#### Ridge Regression Model

In [51]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [52]:
classifier_model = linear_model.RidgeClassifier()

In [59]:
scores = model_selection.cross_val_score(classifier_model, X_train_enc, y_train, cv=3, scoring="f1")
scores

array([0.73412112, 0.73906706, 0.74435543])

In [54]:
# if the model looks good, then let's fit it to the training dataset
classifier_model.fit(X_train_enc, y_train)

RidgeClassifier()

In [55]:
from sklearn.metrics import accuracy_score, f1_score

In [56]:
prediction = classifier_model.predict(X_test_enc)

In [57]:
accuracy_score(prediction, y_test)

0.8046159968165539